In [ ]:
# Login
!az login > /dev/null

In [2]:
# Connect to workspace
from azureml.core import Workspace
ws = Workspace.from_config()

In [4]:
# Pipeline
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment, Environment, RunConfiguration

# Define a PipelineData object to pass data between steps
data_store = ws.get_default_datastore()
prepped_data = PipelineData('prepped',  datastore=data_store)

# Setup Docker environment
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile=f.read()
run_config.environment.docker.base_dockerfile = dockerfile
run_config.environment.python.user_managed_dependencies = True

# Step to run a Python script
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'diabetes_prepare.py',
                         compute_target = 'nossh',
                         runconfig = run_config,
                         # Script arguments include PipelineData
                         arguments = ['--ds-name', 'diabetes',
                                      '--out', prepped_data],
                         # Specify PipelineData as output
                         outputs=[prepped_data])

step2 = PythonScriptStep(name = 'train',
                         source_directory = 'scripts',
                         script_name = 'diabetes_train.py',
                         compute_target = 'nossh',
                         runconfig = run_config,
                         # Script arguments include PipelineData
                         arguments = ['--split-dir', prepped_data],
                         # Specify PipelineData as input
                         inputs=[prepped_data])

# Construct the pipeline
pipeline = Pipeline(workspace = ws, steps = [step1, step2])

# Create an experiment and run the pipeline
experiment = Experiment(workspace = ws, name = 'diabetes-pipeline')
pipeline_run = experiment.submit(pipeline)

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None
Created step prepare data [9dd6af5c][514f9074-68b6-44b7-ae6d-7d9ec5bc9ed3], (This step is eligible to reuse a previous run's output)
Created step train [0d183bf4][88ef328b-77ad-49a7-9d3c-cfe4ed24899d], (This step will run and generate new outputs)
Submitted PipelineRun 1bd19708-269a-4325-8cb8-0f5c1eb79501
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/diabetes-pipeline/runs/1bd19708-269a-4325-8cb8-0f5c1eb79501?wsid=/subscriptions/1bd4017e-d44c-46ab-9e6d-400b8d5f3da8/resourcegroups/ds_cert/workspaces/dscert


In [6]:
# Publish pipeline
published_pipeline = pipeline.publish(name='diabetes',
                                      description='Model training pipeline',
                                      version='1.0')
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/1bd4017e-d44c-46ab-9e6d-400b8d5f3da8/resourceGroups/ds_cert/providers/Microsoft.MachineLearningServices/workspaces/dscert/PipelineRuns/PipelineSubmit/320249d3-cff0-42f7-ab71-55bf26d288c3


In [8]:
# Schedule every day
from azureml.pipeline.core import ScheduleRecurrence, Schedule

daily = ScheduleRecurrence(frequency='Day', interval=1)
pipeline_schedule = Schedule.create(ws, name='Daily Training',
                                        description='trains model every day',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='Training_Pipeline',
                                        recurrence=daily)